In [ ]:
from pathlib import Path
from meteostat import Daily, Point
from tqdm import tqdm
import datetime as dt
import pandas as Pd
import numpy as np
import os

# Automatically find project root (the one containing the 'data' folder)
ROOT = Path.cwd()
if not (ROOT / "data").exists():
    ROOT = ROOT.parent
os.chdir(ROOT)

DATA_DIR = Path("data")
drivers_Path = DATA_DIR/"drivers.csv"
Lap_Times_path = DATA_DIR/"lap_times.csv"
races_Path = DATA_DIR/"races.csv"
circuit_Path = DATA_DIR/"circuits.csv"
qual_Path = DATA_DIR/"qualifying.csv"

drivers = Pd.read_csv(drivers_Path)
lap_times = Pd.read_csv(Lap_Times_path)
races = Pd.read_csv(races_Path)
circuits = Pd.read_csv(circuit_Path)
qual = Pd.read_csv(qual_Path)

drv = drivers.copy()
drv["drivers_name"] = drv["forename"].str.strip() + " " + drv["surname"].str.strip()

best_laps = (
    lap_times
    .groupby(["raceId", "driverId"])["milliseconds"]
    .min()
    .reset_index()
    .rename(columns={"milliseconds":"bestLap_ms"})
)

best_laps["bestLaps_s"] = best_laps["bestLap_ms"]/1000.0
best_laps_named = best_laps.merge(drv[["driverId", "drivers_name"]], on="driverId", how="left")

races_small = races[["raceId", "year", "round", "circuitId", "name", "date"]].copy()
races_small.rename(columns={"name": "gp_name"}, inplace=True)

circuit_small = circuits[["circuitId", "name", "country", "lat", "lng", "alt"]].copy()
circuit_small.rename(columns={"name": "circuit_name"}, inplace=True)

bl = best_laps_named.copy()
bl_race = bl.merge(races_small, on="raceId", how="left")
bl_full = bl_race.merge(circuit_small, on="circuitId", how="left")

def time_str_to_seconds(x):
    """
    Convert time strings like '1:12.345' to seconds (float).
    Handles None/NaN/empty safely.
    """
    if Pd.isna(x) or x == "":
        return np.nan
    s = str(x)
    if ":" in s:
        m, rest = s.split(":")
        return float(m) * 60.0 + float(rest)
    # sometimes it's already seconds-like
    try:
        return float(s)
    except Exception:
        return np.nan

# apply to each q-column
for col in ["q1", "q2", "q3"]:
    if col in qual.columns:
        qual[col + "_s"] = qual[col].apply(time_str_to_seconds)

# best of the available sessions (some drivers only set Q1 or Q2)
q_time_cols = [c for c in ["q1_s","q2_s","q3_s"] if c in qual.columns]
qual["qual_best_s"] = qual[q_time_cols].min(axis=1, skipna=True)

# keep only the fields we need
qual_small = qual[["raceId","driverId","position","qual_best_s"]].copy()
qual_small.rename(columns={"position":"qual_pos"}, inplace=True)

# bl_full currently has: [raceId, driverId, bestLap_s, year, gp_name, circuit_name, country, lat, lng, ...]
bl_q = bl_full.merge(qual_small, on=["raceId","driverId"], how="left")

def get_weather(lat, lng, date_str):
    """
    Fetch weather for given location (lat, lng) and race date.
    Returns a dict with average temperature, rain, wind speed, and pressure.
    """
    try:
        date = Pd.to_datetime(date_str)
        start = date - dt.timedelta(days=1)  # buffer (previous day)
        end = date + dt.timedelta(days=1)
        location = Point(lat, lng)
        data = Daily(location, start, end)
        df = data.fetch()
        if df.empty:
            return {"temp_c": None, "rain_mm": None, "wind_mps": None, "pressure_hpa": None}
        # Take mean of these 3 days for smoother approximation
        return {
            "temp_c": round(df["tavg"].mean(), 2) if "tavg" in df else None,
            "rain_mm": round(df["prcp"].sum(), 2) if "prcp" in df else None,
            "wind_mps": round(df["wspd"].mean(), 2) if "wspd" in df else None,
            "pressure_hpa": round(df["pres"].mean(), 2) if "pres" in df else None,
        }
    except Exception as e:
        print("⚠️ Error:", e)
        return {"temp_c": None, "rain_mm": None, "wind_mps": None, "pressure_hpa": None}

weather_cache = {}
weather_records = []

for _, row in tqdm(bl_q.iterrows(), total=len(bl_q)):
    race_id = row["raceId"]
    if race_id not in weather_cache:
        w = get_weather(row["lat"], row["lng"], row["date"])
        weather_cache[race_id] = w
    weather_records.append(weather_cache[race_id])

weather_df = Pd.DataFrame(weather_records)
bl_weather = Pd.concat([bl_q.reset_index(drop=True), weather_df], axis=1)

bl_weather.to_csv("outputs/f1_features_weather.csv", index=False)


  0%|          | 1/11041 [00:09<28:05:11,  9.16s/it]

⚠️ Error: type NAType doesn't define __round__ method


  0%|          | 20/11041 [00:09<1:02:45,  2.93it/s]

⚠️ Error: type NAType doesn't define __round__ method


  0%|          | 39/11041 [00:10<33:49,  5.42it/s]  

...

⚠️ Error: type NAType doesn't define __round__ method


 81%|████████▏ | 8980/11041 [06:22<01:14, 27.53it/s]

⚠️ Error: type NAType doesn't define __round__ method


100%|██████████| 11041/11041 [07:55<00:00, 23.22it/s]


,raceId,driverId,bestLap_ms,bestLaps_s,drivers_name,year,round,circuitId,gp_name,date,...,country,lat,lng,alt,qual_pos,qual_best_s,temp_c,rain_mm,wind_mps,pressure_hpa
2689,137,11,110886,110.886,Takuma Sato,2002,14,13,Belgian Grand Prix,2002-09-01,...,Belgium,50.4372,5.97139,401,NaN,NaN,NaN,NaN,NaN,NaN
7964,979,840,82830,82.830,Lance Stroll,2017,11,11,Hungarian Grand Prix,2017-07-30,...,Hungary,47.5789,19.24860,264,17.0,79.102,24.53,0.0,7.97,1017.00
3888,194,75,93889,93.889,Shinji Nakano,1998,4,21,San Marino Grand Prix,1998-04-26,...,Italy,44.3439,11.71670,37,NaN,NaN,NaN,NaN,NaN,NaN
4625,231,14,82941,82.941,David Coulthard,1996,8,7,Canadian Grand Prix,1996-06-16,...,Canada,45.5000,-73.52280,13,NaN,NaN,20.70,0.0,6.43,1014.40
2488,127,15,86354,86.354,Jarno Trulli,2002,4,21,San Marino Grand Prix,2002-04-14,...,Italy,44.3439,11.71670,37,NaN,NaN,NaN,NaN,NaN,NaN
5964,872,18,88926,88.926,Jenson Button,2012,13,14,Italian Grand Prix,2012-09-09,...,Italy,45.6156,9.28111,162,2.0,84.133,NaN,NaN,NaN,NaN
8463,1006,4,133811,133.811,Fernando Alonso,2018,18,69,United States Grand Prix,2018-10-21,...,USA,30.1328,-97.64110,161,16.0,95.294,17.20,0.5,13.30,1022.37
10254,1104,840,77769,77.769,Lance Stroll,2023,6,6,Monaco Grand Prix,2023-05-28,...,Monaco,43.7347,7.42056,7,14.0,72.623,19.63,0.0,9.47,1015.63
7319,948,817,88997,88.997,Daniel Ricciardo,2016,1,1,Australian Grand Prix,2016-03-20,...,Australia,-37.8497,144.96800,10,8.0,85.589,16.20,8.1,18.03,1018.43
6678,905,825,80657,80.657,Kevin Magnussen,2014,6,6,Monaco Grand Prix,2014-05-25,...,Monaco,43.7347,7.42056,7,8.0,77.555,17.93,0.3,11.33,1016.03
